In [1]:
import sys
import os

sys.path.append(sys.path[0]+"/tracker")
sys.path.append(sys.path[0]+"/tracker/model")

from track_anything import TrackingAnything
import random 
import string
from dataset.dataset import DAVIS_MO_Test
from tools.runner import run_model_on_davis_set


In [7]:
# Change path to .datasets own folder 
if os.name == 'posix':
    davis_root = '../data.nosync/DAVIS2017'
else:
    davis_root = 'D:\HADA\data\DAVIS'



In [8]:
SAM_checkpoint = "./checkpoints/sam_vit_h_4b8939.pth"
if 'HQ' in sys.prefix.split('.')[-1]:SAM_checkpoint = "./checkpoints/sam_hq_vit_h.pth"
xmem_checkpoint = "./checkpoints/XMem-s012.pth"

In [9]:
runtimeargs_lst = [ 
{
    'DatasetArgs' :{
        'Dataset' : 'Davis',
        'Year' : 17,
        'Set' : 'val',
        'Resolution': '480p'
    },
    'TrackingAnythingArgs' : {
            'use_refinement' : True,
            'refinement_mode' : 'point',
            'addArgs1':'CP'
    }
},{
    'DatasetArgs' :{
        'Dataset' : 'Davis',
        'Year' : 17,
        'Set' : 'val',
        'Resolution': '480p'
    },
    'TrackingAnythingArgs' : {
            'use_refinement' : True,
            'refinement_mode' : 'bbox',
            'addArgs1':'CP'
    }
},{
    'DatasetArgs' :{
        'Dataset' : 'Davis',
        'Year' : 17,
        'Set' : 'val',
        'Resolution': '480p'
    },
    'TrackingAnythingArgs' : {
            'use_refinement' : True,
            'refinement_mode' : 'mask_bbox',
            'addArgs1':'CP'
    }
},{
    'DatasetArgs' :{
        'Dataset' : 'Davis',
        'Year' : 17,
        'Set' : 'val',
        'Resolution': '480p'
    },
    'TrackingAnythingArgs' : {
            'use_refinement' : True,
            'refinement_mode' : 'both',
            'addArgs1':'CP'
    }
},{
    'DatasetArgs' :{
        'Dataset' : 'Davis',
        'Year' : 17,
        'Set' : 'val',
        'Resolution': '480p'
    },
    'TrackingAnythingArgs' : {
            'use_refinement' : True,
            'refinement_mode' : 'both_neg',
            'addArgs1':'CP'
    }
}]
resolution = runtimeargs_lst[0]['DatasetArgs']['Resolution']
year = runtimeargs_lst[0]['DatasetArgs']['Year']
data_set = runtimeargs_lst[0]['DatasetArgs']['Set']
VideoLoader = DAVIS_MO_Test(davis_root, resolution=resolution, imset='20{}/{}.txt'.format(year,data_set), single_object=(year==16))

for runtimeargs in runtimeargs_lst:
    runname = '{}{}{}{}_{}_{}'.format(
    'Refined' if runtimeargs['TrackingAnythingArgs']['use_refinement']  else 'XMEM',
    '_HQ' if 'HQ' in sys.prefix.split('.')[-1] else '_SAM',
    '_'+ runtimeargs['TrackingAnythingArgs']['refinement_mode'] if runtimeargs['TrackingAnythingArgs']['use_refinement'] else '',
    '_' + runtimeargs['TrackingAnythingArgs']['addArgs1'] if runtimeargs['TrackingAnythingArgs']['use_refinement'] and runtimeargs['TrackingAnythingArgs']['addArgs1'] != '' else '',
    'Davis_{}_{}_{}'.format(runtimeargs['DatasetArgs']['Year'],runtimeargs['DatasetArgs']['Set'],runtimeargs['DatasetArgs']['Resolution']) if runtimeargs['DatasetArgs']['Dataset'] == 'Davis' else runtimeargs['DatasetArgs']['Dataset'],
    ''.join(random.choice(string.ascii_letters) for _ in range(5))
    )
    print(f'Running Test: {runname}')
    model = TrackingAnything(SAM_checkpoint, xmem_checkpoint, None,runtimeargs['TrackingAnythingArgs'], save_inner_masks_folder = runname)
    masks, logits, painted_images = run_model_on_davis_set(name = runname,model = model,videoLoader = VideoLoader,compute_metrics = True, save_masks=True, compute_video=True,verbose = False)

Reading file D:\HADA\data\DAVIS\ImageSets\2017/val.txt
Running Test: Refined_SAM_point_CP_Davis_17_val_480p_FTJfO
Initializing BaseSegmenter to cuda:0
Hyperparameters read from the model weights: C^k=64, C^v=512, C^h=64
Single object mode: False
algorithmCP
Sam Refinement ACTIVATED. Mode: point


Tracking image:  66%|██████▌   | 59/90 [00:17<00:09,  3.38it/s]

: 